In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# ===================================================================
# USER SETTINGS
# ===================================================================
fracture_positions = np.array([0, 300, 350, 500, 800, 950, 1010, 1050, 1400, 1675, 1950])
half_length = 250.0           # fracture half-length (each side of well)
W = 2 * half_length             # total y-axis extent of plot = 500 units

num_points_per_interval = 600
num_terms = 120

# Derived
intervals = np.diff(fracture_positions)
total_length = fracture_positions[-1]
num_intervals = len(intervals)
n_vals = np.arange(1, 2 * num_terms + 1, 2)
L_avg = np.mean(intervals)

def pressure_in_interval(x_D_local, t_D_local):
    """Analytical solution between two constant-pressure boundaries"""
    n = n_vals[:, np.newaxis]
    sin_term = np.sin(n * np.pi * x_D_local)
    exp_term = np.exp(-(n * np.pi)**2 * t_D_local)
    sum_term = np.sum((4 / (n * np.pi)) * sin_term * exp_term, axis=0)
    return 1 - sum_term

def plot_at_time(t_D_global=0.01):
    fig, ax = plt.subplots(figsize=(16, 6))
    
    # High-res x-grid
    x_global = np.linspace(0, total_length, num_intervals * num_points_per_interval)
    p_D_global = np.zeros_like(x_global)
    
    midpoint_positions = []
    midpoint_pressures = []
    
    for i in range(num_intervals):
        L_i = intervals[i]
        left = fracture_positions[i]
        right = fracture_positions[i+1]
        mid_x = (left + right) / 2.0
        
        mask = (x_global >= left) & (x_global < right)
        x_local = x_global[mask]
        x_D_local = (x_local - left) / L_i
        
        t_D_local = t_D_global * (L_avg / L_i)**2
        p_interval = pressure_in_interval(x_D_local, t_D_local)
        p_D_global[mask] = p_interval
        
        p_mid = pressure_in_interval(np.array([0.5]), t_D_local)[0]
        midpoint_positions.append(mid_x)
        midpoint_pressures.append(p_mid)
    
    # Create 2D field: y from -250 to +250 (fracture half-lengths)
    y_plot = np.linspace(-half_length, half_length, 160)  # 160 rows for smooth vertical look
    X_plot, Y_plot = np.meshgrid(x_global, y_plot)
    P_2d = np.tile(p_D_global, (len(y_plot), 1))  # pressure is uniform in y (infinite conductivity fracture assumption)
    
    # Plot with custom color scale limits
    im = ax.pcolormesh(X_plot, Y_plot, P_2d, cmap='viridis', vmin=0, vmax=1.0, shading='auto')
    
    # Custom colorbar with descriptive ticks
    cbar = plt.colorbar(im, ax=ax, pad=0.02)
    cbar.set_label('Pressure', fontsize=14)
    cbar.set_ticks([0, 0.5, 1.0])
    cbar.set_ticklabels(['P = Pi', 'P < Pi', 'P << Pi'])
    cbar.ax.tick_params(labelsize=12)
    # Now lets flip the colorbar vertically
    cbar.ax.invert_yaxis()

    # Fractures: full length from -250 to +250
    for pos in fracture_positions:
        ax.axvline(pos, ymin=0, ymax=1, color='red', linewidth=3.5, label='Fracture' if pos == fracture_positions[0] else "")

    # Horizontal wellbore at y=0
    ax.axhline(0, color='whitesmoke', linewidth=6, label='Horizontal Wellbore')

    # Interference indicators at midpoints
    interference_count = 10
    for mid_pos, p_mid in zip(midpoint_positions, midpoint_pressures):
        if p_mid < 0.1:
            ax.axvline(mid_pos, color='cyan', linestyle='--', linewidth=3, alpha=0.9)
            interference_count -= 1
        else:
            ax.axvline(mid_pos, color='k', linestyle=':', linewidth=1.5, alpha=0.75)

    # Styling
    ax.set_ylabel('Fracture Half Length, xf (arbitrary units)', fontsize=14)
    ax.set_xlabel('Lateral Length, Lw (arbitrary units)', fontsize=14)
    flow_regime = "Boundary Dominated Flow" if interference_count == num_intervals else "Infinite Acting" if interference_count == 0 else "Transitional Flow"
    ax.set_title(
        'Multi-Stage Fractured Horizontal Well — Top View\n'
        f'Time Producing = {t_D_global:.5f} (arbitrary units)  │  '
        f'Reservoir Boundaries Reached = {interference_count}/{num_intervals}  │  '
        f'Flow Regime = {flow_regime}',
        fontsize=16, pad=20
    )
    
    # Now lets set the x and y axis tick sizes
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=10)

    # Y-axis ticks showing real distance from well
    ax.set_yticks([-250, -125, 0, 125, 250])
    ax.set_yticklabels(['250', '125', '0 (Well)', '125', '250'])
    ax.grid(True, axis='y', alpha=0.3, color='white', linestyle='--')

    ax.set_xlim(0, total_length)
    ax.set_ylim(-half_length, half_length)

    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='red', lw=4, label='Hydraulic Fracture'),
        Line2D([0], [0], color='white', lw=6, label='Horizontal Wellbore'),
        Line2D([0], [0], color='cyan', linestyle=':', lw=2, label='P = P_i'),
        Line2D([0], [0], color='k', linestyle=':', lw=2, alpha=0.75, label='P < P_i'),
    ]
    # Now lets place the legend in the upper right corner
    ax.legend(handles=legend_elements, loc='upper right', fontsize=11, framealpha=0.75)

    plt.tight_layout()
    plt.show()

# Interactive slider
interact(plot_at_time,
         t_D_global=FloatSlider(min=0.0001, max=0.25, step=0.001, value=0.001,
                            description='Time Producing',
                            continuous_update=False,
                            readout_format='.4f',
                            style={'description_width': 'initial'}))

interactive(children=(FloatSlider(value=0.001, continuous_update=False, description='Time Producing', max=0.25…

<function __main__.plot_at_time(t_D_global=0.01)>